In [3]:
import pandas as pd
import numpy as np
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization import QuadraticProgram

In [8]:
# DataFrame to be used
# The data is processed dataset of the DLRM trace from the Alibaba cloud data-center
csv_path = "filtered_DLRM_trace.csv"

# Parameters:
job_num = 2000   #Number of jobs to be processed
n = 20           #Number of machines
gmax = 8         #Max number of GPUs per machine
cmax = 256       #Max number of CPUs per machine
mem_max = 1024   #Max memory per machine (in MB)
threshold = 0.4  #Threshold for the number of GPUs or CPUs per job to be considered heavy job

df = pd.read_csv(csv_path, usecols=['app_name', 'cpu_request', 'gpu_request', 'memory_request'])

# Filter by jobs with most GPU requests for demonstration
df = df.sort_values(by='gpu_request', ascending=False)

# Select only the first 2000 jobs
df = df.head(job_num)

# Reset the index to normalize it
df = df.reset_index(drop=True)

df["memory_request"] = df["memory_request"].round().astype(int)
print(df)

     app_name  cpu_request  gpu_request  memory_request
0      app_23            8            1              40
1       app_0           12            1             120
2      app_18            8            1              40
3     app_136            8            1              32
4     app_136            8            1              32
...       ...          ...          ...             ...
1995  app_111            8            1              40
1996   app_20            8            1              40
1997   app_22            8            1              40
1998   app_23            8            1              40
1999   app_27            8            1              40

[2000 rows x 4 columns]


In [11]:
alpha = 0.6  # Adjustable variable for priotity calculation

# Create a dictionary to track the appearance count of each app_name
app_count = {}

# Function to calculate priority based on the number of appearances
# The first appearance of an app_name will have a priority of 1, and subsequent appearances will have a priority of alpha^(n-1)
def calculate_priority(app_name):
    if app_name not in app_count:
        app_count[app_name] = 1
        return 1  # First appearance, priority is 1
    else:
        app_count[app_name] += 1
        return alpha ** (app_count[app_name] - 1)  # Subsequent appearances

# Apply the function to calculate priority for each row
df['priority'] = df['app_name'].apply(calculate_priority)

print(df)

     app_name  cpu_request  gpu_request  memory_request      priority
0      app_23            8            1              40  1.000000e+00
1       app_0           12            1             120  1.000000e+00
2      app_18            8            1              40  1.000000e+00
3     app_136            8            1              32  1.000000e+00
4     app_136            8            1              32  6.000000e-01
...       ...          ...          ...             ...           ...
1995  app_111            8            1              40  1.316217e-05
1996   app_20            8            1              40  9.077815e-42
1997   app_22            8            1              40  8.082813e-12
1998   app_23            8            1              40  1.039456e-10
1999   app_27            8            1              40  6.188655e-09

[2000 rows x 5 columns]


In [12]:
print(df['priority'])

0       1.000000e+00
1       1.000000e+00
2       1.000000e+00
3       1.000000e+00
4       6.000000e-01
            ...     
1995    1.316217e-05
1996    9.077815e-42
1997    8.082813e-12
1998    1.039456e-10
1999    6.188655e-09
Name: priority, Length: 2000, dtype: float64


In [19]:
qp = QuadraticProgram()

# Add binary variables for each job-machine combination
# The variable name is in the format "job{i}machine{j}"
# where i is the job index and j is the machine index
for i in range(1, job_num + 1):
    for j in range(1, n + 1):
        qp.binary_var(f"job{i}machine{j}")

linear = {}
# Add objective function to be optimized. This is the total number of jobs assigned to all machines, while
# taking into account the priority of each job.
# The objective function is a linear function of the binary variables, where each variable contributes its priority
for i in range(1, job_num + 1):
    for j in range(1, n + 1):
        linear[f"job{i}machine{j}"] = -df.loc[i - 1, 'priority']
qp.minimize(linear=linear)
#print(qp.prettyprint())

In [20]:
#GPU capacity constraint
# The total number of GPUs assigned to each machine cannot exceed gmax
# The constraint is defined for each machine, where the sum of the GPU requests of all jobs assigned to that machine
# must be less than or equal to gmax
for i in range(1, n + 1):
    machines = {}

    for j in range(1, job_num + 1):
        machines[f"job{j}machine{i}"] = df.loc[j - 1, 'gpu_request']

    qp.linear_constraint(
        linear=machines,
        sense="LE",
        rhs=gmax,
        name=f"gpu_capacity_{i}",
    )

#CPU capacity constraint
# The total number of CPUs assigned to each machine cannot exceed cmax
# The constraint is defined for each machine, where the sum of the CPU requests of all jobs assigned to that machine
# must be less than or equal to cmax
for i in range(1, n + 1):
    machines = {}
    for j in range(1, job_num + 1):
        machines[f"job{j}machine{i}"] = df.loc[j - 1, 'cpu_request']

    qp.linear_constraint(
        linear=machines,
        sense="LE",
        rhs=cmax,
        name=f"cpu_capacity_{i}",
    )

#Memory capacity constraint
# The total memory assigned to each machine cannot exceed mem_max
# The constraint is defined for each machine, where the sum of the memory requests of all jobs assigned to that machine
# must be less than or equal to mem_max
for i in range(1, n + 1):
    machines = {}
    for j in range(1, job_num + 1):
        machines[f"job{j}machine{i}"] = df.loc[j - 1, 'memory_request']

    qp.linear_constraint(
        linear=machines,
        sense="LE",
        rhs=mem_max,
        name=f"memory_capacity_{i}",
    )

# Each job can only be assigned to one machine
# The constraint is defined for each job, where the sum of the binary variables for that job across all machines
# must be equal to 1
# This ensures that each job is assigned to exactly one machine
for i in range(1, job_num + 1):
    linear = {}
    for j in range(1, n + 1):
        linear[f"job{i}machine{j}"] = 1

    qp.linear_constraint(
        linear=linear,
        sense="EQ",
        rhs=1,
        name=f"job_assignment_{i}",
    )

# Two jobs that take more than a threshold of the GPU/CPU/Memory cannot be assigned to the same machine
for i in range(1, job_num + 1):
    for j in range(i + 1, job_num + 1):
        # Check if both jobs are heavy based on GPU, CPU, or memory
        if (
            (df.loc[i - 1, 'gpu_request'] > gmax * 0.3 and df.loc[j - 1, 'gpu_request'] > gmax * threshold) or
            (df.loc[i - 1, 'cpu_request'] > cmax * threshold and df.loc[j - 1, 'cpu_request'] > cmax * threshold) or
            (df.loc[i - 1, 'memory_request'] > mem_max * threshold and df.loc[j - 1, 'memory_request'] > mem_max * threshold)
        ):
            for k in range(1, n + 1):
                qp.linear_constraint(
                    linear={f"job{i}machine{k}": 1, f"job{j}machine{k}": 1},
                    sense="LE",
                    rhs=1,
                    name=f"conflict_{i}_{j}_{k}",
                )

#print(qp.prettyprint())


In [ ]:
# Instantiating the Qiskit Minimum Eigensolver with Sampler and COBYLA optimizer
# The Qiskit Minimum Eigensolver is used to solve the quadratic program (QP) using the QAOA algorithm
# The Sampler is used to sample the solutions, and the COBYLA optimizer is used to optimize the parameters
# The initial point for the optimizer is set to [0.0, 0.0]
algorithm_globals.random_seed = 42  # Set a random seed for reproducibility
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), initial_point=[0.0, 0.0])
qaoa = MinimumEigenOptimizer(qaoa_mes)
qaoa_result = qaoa.solve(qp)
print(qaoa_result.prettyprint())